In [ ]:
import random

import numpy as np

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense

from tensorflow.keras import backend as K

In [ ]:
from tensorflow.keras.datasets import mnist
(train_digits, train_labels), (test_digits, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# some variables...
image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  # we have grayscale images
# NOTE: image_height == image_width == 28

# re-shape the images data
train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

# re-scale the image data to values between (0.0,1.0]
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# one-hot encode the labels - we have 10 output classes
# so 3 -> [0 0 0 1 0 0 0 0 0 0], 5 -> [0 0 0 0 0 1 0 0 0 0] & so on
from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

((60000, 10), (10000, 10))

In [ ]:
num_clients = 10
size = train_data.shape[0] // num_clients
from collections import defaultdict
clients_dict = defaultdict(dict)
client_no = 1
for i in range(0,size*num_clients, size):
  clients_dict["client_"+str(client_no)] = {"X": train_data[i: i+size], "y": train_labels_cat[i: i+size]}
  client_no+=1

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Flatten(input_shape=(28, 28, 1)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        return model

In [ ]:
#create optimizer
comms_round = 10
lr = 0.01 
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)

In [ ]:
def weight_scalling_factor(clients_trn_data, curr_client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = clients_trn_data[curr_client_name]["X"].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([clients_trn_data[client_name]["X"].size for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = clients_trn_data[curr_client_name]["X"].size * bs
    return local_count/global_count

In [ ]:
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
    
        import pdb; pdb.set_trace()
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

In [ ]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
#commence global training loop
comms_round = 10
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_dict.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_dict[client]["X"], clients_dict[client]["y"], epochs=15, verbose=0) 
        
        #scale the model weights and add to list
        
        scaling_factor = weight_scalling_factor(clients_dict, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    global_acc, global_loss = test_model(test_data, test_labels_cat,  global_model, comm_round)

> <ipython-input-8-c383bd4cd182>(18)sum_scaled_weights()
-> layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
(Pdb) type(grad_list_tuple)
<class 'tuple'>
(Pdb) type(grad_list_tuple[0])
<class 'numpy.ndarray'>
(Pdb) type(grad_list_tuple[1])
<class 'numpy.ndarray'>
(Pdb) grad_list_tuple[1].shape
(784, 128)
(Pdb) grad_list_tuple[0].shape
(784, 128)
(Pdb) n
> <ipython-input-8-c383bd4cd182>(19)sum_scaled_weights()
-> avg_grad.append(layer_mean)
(Pdb) layer_mean
<tf.Tensor: shape=(784, 128), dtype=float32, numpy=
array([[ 0.02402751,  0.05698669, -0.02011852, ..., -0.01844793,
        -0.03818075, -0.03779719],
       [-0.07331424, -0.04113057,  0.0444822 , ..., -0.00572068,
         0.03491629,  0.04130774],
       [ 0.04353614, -0.06092962,  0.01644968, ...,  0.0261076 ,
         0.02592262, -0.06144632],
       ...,
       [ 0.04288005,  0.06939675,  0.06672618, ...,  0.05533021,
         0.04333788,  0.04640239],
       [ 0.00245635, -0.02440473, -0.01795647, ...,  0.03528569,
   

In [ ]:
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(train_data, train_labels_cat, epochs=15, verbose=1)

Epoch 1/15
1875/1875 [==============================] - 3s 1ms/step - loss: 2.0023 - accuracy: 0.3889
Epoch 2/15
1875/1875 [==============================] - 3s 1ms/step - loss: 1.5154 - accuracy: 0.6931
Epoch 3/15
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1845 - accuracy: 0.7668
Epoch 4/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.9735 - accuracy: 0.8023
Epoch 5/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.8385 - accuracy: 0.8205
Epoch 6/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.7474 - accuracy: 0.8341
Epoch 7/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6825 - accuracy: 0.8430
Epoch 8/15
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6340 - accuracy: 0.8502
Epoch 9/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5964 - accuracy: 0.8564
Epoch 10/15
1875/1875 [==============================] - 3s 2ms/step - loss: 0.566

In [ ]:
#test the SGD global model and print out metrics
SGD_acc, SGD_loss = test_model(test_data, test_labels_cat, SGD_model, 1)

comm_round: 1 | global_acc: 88.640% | global_loss: 1.70375394821167
